### 1) Szükséges könyvtárak installálása, betöltése ⛽

In [41]:
!pip install pandas bertopic numpy

In [4]:
import numpy as np
import pandas as pd

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import CountVectorizer

### 2) Adat kiválasztása 📊

❓ Melyik portál legyen?
- KurucInfo (kód: `kuruc`)
- Népszava (kód: `nepszava`)
- Origo (kód: `origo`)
- Magyar Nemzet (kód: `mno`)

In [21]:
portal = 'nepszava'  # <-- ide írd a portál kódot

❓ Melyik év legyen?
- Nézd meg a http://porthos.elte-dh.hu:8000/ oldalon az elérhető JSON fájlokat

In [22]:
year = '2013'  #  <-- ide írd az évet

❗ A portálnevet és az évet összerakva már le tudod szedni az adatot a szerverről:

In [23]:
server_url = f'http://porthos.elte-dh.hu:8000/{portal}_{year}.json'  # ezzel megcsináljuk a szerver URL-t

❗Ez a kód leszedi a szerverről az adatot és berakja egy pandas DataFrame-be.

In [24]:
df = pd.read_json(server_url)  # Kicsit várni kell.

In [42]:
# Így néz ki az adatunk
df.head(5)

### 3) BERTopic inicializálása 🏁

❗Először kiválasztjuk azokat a szavakat, amikre nincs szükség. Ha valamelyik mégis kellene, akkor nyugodtan töröld ki!

In [15]:
hu_stop_words = [
    'a', 'abban', 'ahhoz', 'ahogy', 'ahol', 'aki', 'akik', 'akkor', 'alatt', 'amely', 'amelyek', 'amelyekben', 'amelyeket', 'amelyet',
    'amelynek', 'ami', 'amikor', 'amit', 'amolyan', 'amíg', 'annak', 'arra', 'arról', 'az', 'azok',
    'azon', 'azonban', 'azt', 'aztán', 'azután', 'azzal', 'azért', 'be', 'belül', 'benne', 'bár','csak', 'de', 'e',
    'ebben', 'eddig', 'egy', 'egyes', 'egyetlen', 'egyik', 'egyre', 'egyéb', 'egész', 'ehhez', 'ekkor', 'el', 'ellen', 'elsõ', 'elég',
    'elõ', 'elõször', 'elõtt', 'emilyen', 'ennek', 'erre', 'ez', 'ezek', 'ezen', 'ezt', 'ezzel', 'ezért', 'fel', 'felé', 'hanem',
    'hiszen', 'hogy', 'hogyan', 'igen', 'ill', 'ill.', 'illetve', 'ilyen', 'ilyenkor', 'ismét', 'ison', 'itt', 'jobban', 'jó', 'jól', 'kell', 'kellett',
    'keressünk', 'keresztül', 'ki', 'kívül', 'között', 'közül', 'legalább', 'legyen', 'lehet', 'lehetett', 'lenne', 'lenni', 'lesz', 'lett', 'maga', 'magát', 'majd',
    'majd', 'meg', 'mellett', 'mely', 'melyek', 'mert', 'mi', 'mikor', 'milyen', 'minden', 'mindenki', 'mindent', 'mindig', 'mint', 'mintha', 'mit', 'mivel', 'miért', 'most',
    'már', 'más', 'másik', 'még', 'míg', 'nagy', 'nagyobb', 'nagyon', 'ne', 'nekem', 'neki', 'nem', 'nincs', 'néha', 'néhány', 'nélkül', 'olyan', 'ott', 'pedig',
    'persze', 'rá', 's', 'saját', 'sem', 'semmi', 'sok', 'sokat', 'sokkal', 'szemben', 'szerint', 'szinte', 'számára', 'talán', 'tehát', 'teljes', 'tovább', 'továbbá',
    'több', 'ugyanis', 'utolsó', 'után', 'utána', 'vagy', 'vagyis', 'vagyok', 'valaki', 'valami', 'valamint', 'való', 'van', 'vannak', 'vele', 'vissza', 'viszont', 'volna',
    'volt', 'voltak', 'voltam', 'voltunk', 'által', 'általában', 'át', 'én', 'éppen', 'és', 'így', 'õ', 'õk', 'õket', 'össze', 'úgy', 'új', 'újabb', 'újra'
    ]

❗ Utána kell egy model ami a deskriptív szavakért lesz felelős. A CountVectorizer egy olyan eszköz, amely a dokumentumokban található szavak előfordulásának számát veszi figyelembe, hogy azok alapján jellemezze és csoportosítsa a témákat.

In [26]:
# Itt meg lehet változtatni a min_df-et is és az ngram_range-et is (pl 1, 3 -ra)
vectorizer_model = CountVectorizer(stop_words=hu_stop_words, min_df=2, ngram_range=(1, 2))

❗Végül csinálunk egy bertopic topic model objektumot. Ha a nyelv magyar akkor egy többnyelvű nyelvmodellt használ. A min_topic_size-ot nyugodtan változtasd ha nem tetszik a kiement.

In [27]:
topic_model = BERTopic(language='hungarian', vectorizer_model=vectorizer_model, min_topic_size=15)

### 4) A modellnek megadjuk az adatokat, és futtatjuk. ⌛
... ez eltart egy ideig ...

In [28]:
topics, probs = topic_model.fit_transform(df['first_paragraph'], embeddings=np.array(list(df['embedding'])))

### 5) Végül vizualizálhatjuk az eredményeinket 📈

In [43]:
# Topikok elhelyezkedése egy 2D-s térben
topic_model.visualize_topics()

In [44]:
# Topik számok és a hozzájuk tartozó szavak relevanciája
topic_model.visualize_barchart(top_n_topics=100)

### 6) Az eredményeink alapján visszatérhetünk az adatunkhoz, és megnézhetjük, mi került egy topikba. 👀

In [35]:
# Itt hozzárendeljük a topik ereményeket az adatunkhoz
no_embeddings_df = df.drop('embedding', axis=1)
no_embeddings_df['topic'] = topics

In [45]:
topic_n = 2  # <-- ide írd a topik számot ami érdekel
no_embeddings_df[no_embeddings_df['topic'] == topic_n]